In [1]:
import os
# Find the latest version of spark 3.0  from http://www.apache.org/dist/spark/ and enter as the spark version
spark_version = 'spark-3.0.3'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Hit:1 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:2 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:3 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Get:4 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:5 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Get:6 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Ign:7 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:8 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease [1,581 B]
Hit:9 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:11 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease [15.9 kB]
Hit:12 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Get:13 http

In [2]:
!wget https://jdbc.postgresql.org/download/postgresql-42.2.9.jar

--2022-08-12 03:01:29--  https://jdbc.postgresql.org/download/postgresql-42.2.9.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 914037 (893K) [application/java-archive]
Saving to: ‘postgresql-42.2.9.jar’

postgresql-42.2.9.j 100%[===================>] 892.61K  5.69MB/s    in 0.2s    

2022-08-12 03:01:30 (5.69 MB/s) - ‘postgresql-42.2.9.jar’ saved [914037/914037]



In [3]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("CloudETL").config("spark.driver.extraClassPath","/content/postgresql-42.2.9.jar").getOrCreate()

In [4]:
from pyspark import SparkFiles
# Load in dataset from S3 into a DataFrame
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Kitchen_v1_00.tsv.gz"
spark.sparkContext.addFile(url)

kitchen_df = spark.read.option('header', 'true').csv(SparkFiles.get("amazon_reviews_us_Kitchen_v1_00.tsv.gz"), inferSchema=True, sep='\t', timestampFormat="mm/dd/yy")
kitchen_df.show(10)

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   37000337|R3DT59XH7HXR9K|B00303FI0G|     529320574|Arthur Court Pape...|         Kitchen|          5|            0|          0|   N|                Y|Beautiful. Looks ...|Beautiful.  Looks...| 2015-08-31|
|         US|   15272914|R1LFS11BNASSU8|B00JCZKZN6|     274237558|Olde Thompson Bav...|         Kitchen|          5|    

In [5]:
print(f'Total kitchen reviews: {kitchen_df.count()}')

Total kitchen reviews: 4880466


In [6]:
from pyspark.sql.functions import *

# Make kitchen review id dataframe and convert review date variable to date
kitchen_review_ids = kitchen_df.select(col("review_id"),col("customer_id"),col("product_id"),
                               col("product_parent"),to_date(col("review_date"),"yyyy-mm-dd").alias('review_date'))

In [7]:
# Verify datatypes
kitchen_review_ids.dtypes

[('review_id', 'string'),
 ('customer_id', 'int'),
 ('product_id', 'string'),
 ('product_parent', 'int'),
 ('review_date', 'date')]

In [8]:
# View table
kitchen_review_ids.show(10)

+--------------+-----------+----------+--------------+-----------+
|     review_id|customer_id|product_id|product_parent|review_date|
+--------------+-----------+----------+--------------+-----------+
|R3DT59XH7HXR9K|   37000337|B00303FI0G|     529320574| 2015-01-31|
|R1LFS11BNASSU8|   15272914|B00JCZKZN6|     274237558| 2015-01-31|
|R296RT05AG0AF6|   36137863|B00JLIKA5C|     544675303| 2015-01-31|
|R3V37XDZ7ZCI3L|   43311049|B000GBNB8G|     491599489| 2015-01-31|
|R14GU232NQFYX2|   13763148|B00VJ5KX9S|     353790155| 2015-01-31|
| RZQH4V7L2O1PL|   19009420|B00HYB5YY0|     432241873| 2015-01-31|
|R1F8JMOSPJ3KO7|   40599388|B000HEBAV2|     584680984| 2015-01-31|
|R1ZISGY2BWW4Z5|   22719359|B0012DS4GG|     772637306| 2015-01-31|
|R17PW4I3AE5WZW|   47478640|B00FLQ4EE6|     755416578| 2015-01-31|
|R3D93G1KTP6A8P|   34195504|B00DBS9OTG|     648762742| 2015-01-31|
+--------------+-----------+----------+--------------+-----------+
only showing top 10 rows



In [9]:
# Make kitchen products table
kitchen_products = kitchen_df[['product_id','product_title']]

In [10]:
# Check number of rows
print(f'Number of rows before duplicates removed: {kitchen_products.count()}')

# Drop duplicate rows
kitchen_products = kitchen_products.drop_duplicates()

# Check number of rows after duplicates removed 
print(f'Number of rows after duplicates removed: {kitchen_products.count()}')

Number of rows before duplicates removed: 4880466
Number of rows after duplicates removed: 433478


In [11]:
# Create kitchen customers table and count number of times each customer has left a review
kitchen_customers = kitchen_df.groupby("customer_id").count()

In [12]:
# View table
kitchen_customers.show()

+-----------+-----+
|customer_id|count|
+-----------+-----+
|   23042837|    1|
|   48875707|    2|
|   48088660|    1|
|   20781887|    6|
|    1075003|    1|
|   28377689|    3|
|    8338749|    1|
|   39271457|    6|
|   38209321|    5|
|   51397605|    1|
|   45337932|    7|
|   12948675|    3|
|   38892468|    1|
|   43582450|    1|
|   13156755|    1|
|   13433330|    1|
|   17067926|    7|
|   13780617|    1|
|   50607818|    1|
|   33437759|    2|
+-----------+-----+
only showing top 20 rows



In [13]:
# Rename count column
kitchen_customers = kitchen_customers.withColumnRenamed('count','customer_count')
kitchen_customers.show()

+-----------+--------------+
|customer_id|customer_count|
+-----------+--------------+
|   23042837|             1|
|   48875707|             2|
|   48088660|             1|
|   20781887|             6|
|    1075003|             1|
|   28377689|             3|
|    8338749|             1|
|   39271457|             6|
|   38209321|             5|
|   51397605|             1|
|   45337932|             7|
|   12948675|             3|
|   38892468|             1|
|   43582450|             1|
|   13156755|             1|
|   13433330|             1|
|   17067926|             7|
|   13780617|             1|
|   50607818|             1|
|   33437759|             2|
+-----------+--------------+
only showing top 20 rows



In [14]:
# Configuration for RDS instance
mode="append"
jdbc_url = "jdbc:postgresql://reviewsdb.csumjblrdtra.us-west-2.rds.amazonaws.com:5432/reviews_db"
config = {"user":"<user>",
          "password": "<password>",
          "driver":"org.postgresql.Driver"}

In [15]:
# Write kitchen_review_ids to table
kitchen_review_ids.write.jdbc(url=jdbc_url, table='kitchen_review_id_table', mode=mode, properties=config)

In [16]:
# Write kitchen_products to table
kitchen_products.write.jdbc(url=jdbc_url, table='kitchen_products', mode=mode, properties=config)

In [17]:
# Write kitchen_customers to table
kitchen_customers.write.jdbc(url=jdbc_url, table='kitchen_customers', mode=mode, properties=config)